# Libraries

In [3]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from keras_preprocessing.sequence import pad_sequences

import numpy as np
import os

2024-03-29 21:07:29.671639: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-03-29 21:07:30.071924: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-29 21:07:30.071973: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-29 21:07:30.134362: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-03-29 21:07:30.268078: I tensorflow/core/platform/cpu_feature_guar

# Preprocessing

In [13]:
# base path
directory_path = './labels_final'
# current directory
c_dir = os.getcwd()

# all actions
actions = np.array(sorted([folder for folder in os.listdir(directory_path) if os.path.isdir(os.path.join(directory_path, folder))])) # sorted to follow folder arrangement

# specific actions
# actions = np.array(['alligator', 'flower', 'kiss', 'listen', 'orange'])
# actions = np.array(['afternoon', 'house', 'again', 'open', 'kiss', 'sorry'])
# actions = np.array(sorted([folder for folder in os.listdir('./labels_new') if os.path.isdir(os.path.join(directory_path, folder))]))

In [14]:
# create a dictionary for int representation of actions
label_map = {label:num for num, label in enumerate(actions)}
label_map

{'all': 0,
 'base': 1,
 'bye': 2,
 'close': 3,
 'down': 4,
 'for': 5,
 'good': 6,
 'have': 7,
 'hello': 8,
 'how': 9,
 'if': 10,
 'listen': 11,
 'mad': 12,
 'nap': 13,
 'no': 14,
 'noisy': 15,
 'now': 16,
 'please': 17,
 'quiet': 18,
 'sad': 19,
 'show': 20,
 'thankyou': 21,
 'time': 22,
 'we': 23,
 'will': 24,
 'work': 25}

Note that at this point, we will not access the video folder, only the numpy folder.

In [15]:
sequences, labels = [], []  # sequence -> video, labels -> action
for action in actions:
    no_actions = len(os.listdir(os.path.join(directory_path, action)))
    action_counter = 0
    print('Opening path:', os.path.join(directory_path, action))
    print(f'Number of instances: {no_actions}')
    for num in range(1, no_actions + 1):
        window = []         # window -> single frame
        file = str(action) + "_" + str(num)
        no_frames_per_action = len(os.listdir(os.path.join(directory_path, action, file)))
        print(f'Number of frames in {file}: {no_frames_per_action}')
        if no_frames_per_action >= 15 and action_counter < 150:
            action_counter += 1
            for frame_num in range(1, no_frames_per_action + 1):
                res = np.load(os.path.join(directory_path, action, file,  "{}.npy".format(frame_num)))     # res -> coordinate key points
                window.append(res)
            sequences.append(window)
            labels.append(label_map[action])
    print('-'*75)

Opening path: ./labels_final/all
Number of instances: 386
Number of frames in all_1: 39
Number of frames in all_2: 33
Number of frames in all_3: 25
Number of frames in all_4: 11
Number of frames in all_5: 4
Number of frames in all_6: 7
Number of frames in all_7: 33
Number of frames in all_8: 87
Number of frames in all_9: 6
Number of frames in all_10: 29
Number of frames in all_11: 9
Number of frames in all_12: 133
Number of frames in all_13: 68
Number of frames in all_14: 11
Number of frames in all_15: 6
Number of frames in all_16: 46
Number of frames in all_17: 16
Number of frames in all_18: 18
Number of frames in all_19: 10
Number of frames in all_20: 56
Number of frames in all_21: 54
Number of frames in all_22: 9
Number of frames in all_23: 192
Number of frames in all_24: 31
Number of frames in all_25: 26
Number of frames in all_26: 30
Number of frames in all_27: 6
Number of frames in all_28: 33
Number of frames in all_29: 6
Number of frames in all_30: 55
Number of frames in all_31:

In [ ]:
# due to difference in number of frames, pad x and y
x = np.array(pad_sequences(sequences, dtype = 'float', padding = 'post', value = 0))
y = to_categorical(labels).astype(int)

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, stratify = y)

In [ ]:
input_shape = (x_train.shape[1], x_train.shape[2])
print(x_train.shape)
print(x_test.shape)

(3116, 321, 225)
(779, 321, 225)


## Quicksave

In [7]:
np.save('x_train.npy', x_train)
np.save('x_test.npy', x_test)
np.save('y_train.npy', y_train)
np.save('y_test.npy', y_test)

## Reload Pre-saved

In [1]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from keras_preprocessing.sequence import pad_sequences

import numpy as np
import os

# base path
directory_path = './labels_final'
# current directory
c_dir = os.getcwd()

# all actions
actions = np.array(sorted([folder for folder in os.listdir(directory_path) if os.path.isdir(os.path.join(directory_path, folder))])) # sorted to follow folder arrangement

x_train = np.load('x_train.npy')
x_test = np.load('x_test.npy')
y_train = np.load('y_train.npy')
y_test = np.load('y_test.npy')

input_shape = (x_train.shape[1], x_train.shape[2])

In [2]:
np.sum(y_train, axis = 0)

array([120, 120, 120, 120, 116, 120, 120, 120, 120, 120, 120, 120, 120,
       120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120])

# Models

## LSTM

In [2]:
import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Bidirectional
from tensorflow.keras.callbacks import TensorBoard, TerminateOnNaN, EarlyStopping, ModelCheckpoint

from sklearn.metrics import accuracy_score

In [4]:
# for logging of data with TensorBoard
log_dir = os.path.join(c_dir, 'Logs')
tb_callback = TensorBoard(log_dir = log_dir)

# to end training when failure happens ie. loss == nan
term = TerminateOnNaN()

# to stop training early if there is no change in loss
early = EarlyStopping(monitor = 'val_loss', patience = 200)

# regulariser 
l2_strength = 0.001

In [5]:
def choose_lstm(n):
    if n == 1:
        # normal lstm layers with dropout, performance was not ideal
        model = Sequential()
        model.add(LSTM(128, return_sequences = True, input_shape = input_shape))
        model.add(Dropout(0.1))
        model.add(LSTM(64, return_sequences = False))
        model.add(Dropout(0.1))
        model.add(Dense(actions.shape[0], activation = "softmax"))

        return model

    elif n == 2:
        # follow up on initial model, include more lstm and dense layers
        model = Sequential()
        model.add(LSTM(64, return_sequences = True, input_shape = input_shape))
        model.add(LSTM(128, return_sequences = True))
        model.add(LSTM(64, return_sequences = False))
        model.add(Dense(64))
        model.add(Dense(32))
        model.add(Dense(8))
        model.add(Dense(actions.shape[0], activation = "softmax"))

        return model

    elif n == 3:
        # learned about bidirectionality and its applications in nlp, performance improved significantly with nlp
        model = Sequential()
        model.add(LSTM(128, return_sequences = True, input_shape = input_shape))
        model.add(Dropout(0.5)) # heavier dropout after first hidden layer
        model.add(Bidirectional(LSTM(64)))
        model.add(Dropout(0.2))
        model.add(Dense(actions.shape[0], activation = "softmax"))

        return model

    elif n == 4:
        # increased number of bidirectional lstm layers
        model = Sequential()
        model.add(LSTM(128, return_sequences = True, input_shape = input_shape))
        model.add(Dropout(0.5))
        model.add(Bidirectional(LSTM(64, return_sequences = True)))
        model.add(Dropout(0.3))
        model.add(Bidirectional(LSTM(32)))
        model.add(Dropout(0.2))
        model.add(Dense(actions.shape[0], activation = "softmax"))

        return model

    elif n == 5:
        # introduced regularisation in last bidirectinal lstm layer
        model = Sequential()
        model.add(LSTM(128, return_sequences = True, input_shape = input_shape))
        model.add(Dropout(0.5))
        model.add(Bidirectional(LSTM(64, return_sequences = True)))
        model.add(Dropout(0.3))
        model.add(Bidirectional(LSTM(32, kernel_regularizer = keras.regularizers.L2(l2_strength))))
        model.add(Dropout(0.2))
        model.add(Dense(actions.shape[0], activation = "softmax"))

        return model
    
    elif n == 6:
        # step back and applied regularisation to simpler model
        model = Sequential()
        model.add(LSTM(128, return_sequences = True, input_shape = input_shape))
        model.add(Dropout(0.5))
        model.add(Bidirectional(LSTM(64, kernel_regularizer = keras.regularizers.L2(l2_strength))))
        model.add(Dropout(0.2))
        model.add(Dense(actions.shape[0], activation = "softmax"))

        return model

## Base Model

The base model serves as the best model through trial and error. Using this base model, we will adjust certain parameters and test these variations.

In [7]:
model1 = choose_lstm(3)
opt = keras.optimizers.Adam(learning_rate = 0.001)
model1.compile(optimizer = opt, loss = "categorical_crossentropy", metrics = ['categorical_accuracy'])
model1.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_2 (LSTM)               (None, 321, 128)          181248    
                                                                 
 dropout_1 (Dropout)         (None, 321, 128)          0         
                                                                 
 bidirectional_1 (Bidirecti  (None, 128)               98816     
 onal)                                                           
                                                                 
 dropout_2 (Dropout)         (None, 128)               0         
                                                                 
 dense (Dense)               (None, 26)                3354      
                                                                 
Total params: 283418 (1.08 MB)
Trainable params: 283418 (1.08 MB)
Non-trainable params: 0 (0.00 Byte)
__________________

In [8]:
model1.fit(x_train, y_train, epochs = 500, batch_size = 64, validation_split = 0.2, callbacks = [term, tb_callback, early])

Epoch 1/500


2024-03-29 18:18:27.285507: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8904
2024-03-29 18:18:27.949083: I external/local_xla/xla/service/service.cc:168] XLA service 0x7f93879df950 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-03-29 18:18:27.949117: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 3060 Laptop GPU, Compute Capability 8.6
2024-03-29 18:18:27.961693: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1711707508.052116  135287 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


39/39 [==============================] - 7s 82ms/step - loss: 3.0345 - categorical_accuracy: 0.1091 - val_loss: 2.8171 - val_categorical_accuracy: 0.1362
Epoch 2/500
39/39 [==============================] - 3s 67ms/step - loss: 2.7315 - categorical_accuracy: 0.1461 - val_loss: 2.6715 - val_categorical_accuracy: 0.1875
Epoch 3/500
39/39 [==============================] - 2s 58ms/step - loss: 2.6171 - categorical_accuracy: 0.1794 - val_loss: 2.6715 - val_categorical_accuracy: 0.1779
Epoch 4/500
39/39 [==============================] - 2s 59ms/step - loss: 2.5741 - categorical_accuracy: 0.1902 - val_loss: 2.6023 - val_categorical_accuracy: 0.1715
Epoch 5/500
39/39 [==============================] - 2s 58ms/step - loss: 2.5419 - categorical_accuracy: 0.1966 - val_loss: 2.5879 - val_categorical_accuracy: 0.1763
Epoch 6/500
39/39 [==============================] - 2s 60ms/step - loss: 2.4789 - categorical_accuracy: 0.2291 - val_loss: 2.5300 - val_categorical_accuracy: 0.2003
Epoch 7/500
39/3

In [9]:
# take model predictions
res1 = model1.predict(x_test)

25/25 [==============================] - 1s 20ms/step


In [10]:
y_true = np.argmax(y_test, axis = 1).tolist()
y_pred1 = np.argmax(res1, axis = 1).tolist()

In [11]:
print(y_true)
print(y_pred1)

[23, 16, 21, 12, 22, 24, 5, 7, 18, 6, 18, 15, 22, 19, 22, 18, 6, 13, 17, 6, 5, 4, 16, 8, 19, 9, 12, 3, 10, 0, 8, 18, 23, 25, 15, 4, 0, 8, 9, 9, 0, 1, 15, 1, 18, 20, 24, 24, 1, 7, 2, 24, 17, 23, 0, 18, 10, 25, 22, 16, 23, 15, 12, 1, 18, 16, 7, 18, 17, 19, 16, 8, 2, 15, 24, 24, 13, 1, 24, 7, 2, 19, 10, 19, 23, 10, 20, 18, 3, 5, 7, 13, 3, 22, 21, 22, 21, 16, 15, 5, 22, 18, 6, 19, 16, 9, 6, 1, 20, 24, 13, 12, 11, 10, 8, 15, 10, 12, 3, 17, 14, 25, 13, 4, 4, 22, 2, 12, 15, 21, 5, 12, 6, 16, 8, 5, 1, 11, 19, 24, 7, 0, 6, 0, 20, 1, 15, 7, 16, 3, 6, 15, 5, 18, 12, 23, 4, 6, 23, 4, 3, 12, 4, 9, 23, 19, 15, 5, 0, 12, 20, 5, 16, 13, 11, 12, 0, 5, 17, 12, 2, 6, 6, 20, 22, 8, 13, 20, 15, 15, 19, 21, 7, 22, 24, 12, 11, 12, 6, 14, 18, 4, 15, 4, 6, 6, 18, 1, 23, 5, 4, 16, 15, 7, 11, 1, 14, 23, 19, 17, 0, 16, 14, 25, 11, 2, 22, 10, 3, 2, 12, 7, 14, 22, 7, 1, 25, 13, 19, 7, 17, 9, 9, 0, 3, 23, 11, 1, 17, 7, 2, 10, 11, 22, 5, 16, 17, 23, 3, 23, 15, 15, 9, 18, 10, 5, 19, 11, 25, 6, 22, 4, 8, 11, 1, 14, 3, 

In [12]:
print(f'Accuracy: {round(accuracy_score(y_true, y_pred1)*100, 1)}%')

if accuracy_score(y_true, y_pred1) >= 0.70:
    model1.save('lstm_model1.h5', save_format = 'h5')
else:
    print('Model accuracy insufficient')

Accuracy: 74.8%


/home/zhwei/anaconda3/envs/slproj/lib/python3.9/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


### Variant 1

In [13]:
model2 = choose_lstm(4)
opt = keras.optimizers.Adam(learning_rate = 0.001)
model2.compile(optimizer = opt, loss = "categorical_crossentropy", metrics = ['categorical_accuracy'])
model2.summary()
model2.fit(x_train, y_train, epochs = 500, batch_size = 64, validation_split = 0.2, callbacks = [term, tb_callback, early])

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_4 (LSTM)               (None, 321, 128)          181248    
                                                                 
 dropout_3 (Dropout)         (None, 321, 128)          0         
                                                                 
 bidirectional_2 (Bidirecti  (None, 321, 128)          98816     
 onal)                                                           
                                                                 
 dropout_4 (Dropout)         (None, 321, 128)          0         
                                                                 
 bidirectional_3 (Bidirecti  (None, 64)                41216     
 onal)                                                           
                                                                 
 dropout_5 (Dropout)         (None, 64)               

In [14]:
# take model predictions
res2 = model2.predict(x_test)
y_true = np.argmax(y_test, axis = 1).tolist()
y_pred2 = np.argmax(res2, axis = 1).tolist()
print(f'Accuracy: {round(accuracy_score(y_true, y_pred2)*100, 1)}%')

if accuracy_score(y_true, y_pred2) >= 0.70:
    model2.save('lstm_model2.h5', save_format = 'h5')
else:
    print('Model accuracy insufficient')

25/25 [==============================] - 2s 30ms/step
Accuracy: 73.7%


/home/zhwei/anaconda3/envs/slproj/lib/python3.9/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


### Variant 2

In [8]:
model3 = choose_lstm(5)
opt = keras.optimizers.Adam(learning_rate = 0.001)
model3.compile(optimizer = opt, loss = "categorical_crossentropy", metrics = ['categorical_accuracy'])
model3.summary()
model3.fit(x_train, y_train, epochs = 500, batch_size = 64, validation_split = 0.2, callbacks = [term, tb_callback, early])

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_3 (LSTM)               (None, 321, 128)          181248    
                                                                 
 dropout_3 (Dropout)         (None, 321, 128)          0         
                                                                 
 bidirectional_2 (Bidirecti  (None, 321, 128)          98816     
 onal)                                                           
                                                                 
 dropout_4 (Dropout)         (None, 321, 128)          0         
                                                                 
 bidirectional_3 (Bidirecti  (None, 64)                41216     
 onal)                                                           
                                                                 
 dropout_5 (Dropout)         (None, 64)               

2024-03-29 19:23:00.101008: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8904
2024-03-29 19:23:01.084071: I external/local_xla/xla/service/service.cc:168] XLA service 0x7f71300b8fc0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-03-29 19:23:01.084105: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 3060 Laptop GPU, Compute Capability 8.6
2024-03-29 19:23:01.105478: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1711711381.221093  178610 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


39/39 [==============================] - 10s 134ms/step - loss: 3.3434 - categorical_accuracy: 0.0995 - val_loss: 3.1267 - val_categorical_accuracy: 0.0978
Epoch 2/500
39/39 [==============================] - 3s 88ms/step - loss: 2.9653 - categorical_accuracy: 0.1541 - val_loss: 2.8510 - val_categorical_accuracy: 0.1811
Epoch 3/500
39/39 [==============================] - 3s 88ms/step - loss: 2.7348 - categorical_accuracy: 0.1922 - val_loss: 2.7206 - val_categorical_accuracy: 0.1811
Epoch 4/500
39/39 [==============================] - 3s 86ms/step - loss: 2.7185 - categorical_accuracy: 0.1794 - val_loss: 2.7318 - val_categorical_accuracy: 0.1651
Epoch 5/500
39/39 [==============================] - 3s 86ms/step - loss: 2.6726 - categorical_accuracy: 0.1822 - val_loss: 2.6693 - val_categorical_accuracy: 0.1763
Epoch 6/500
39/39 [==============================] - 3s 87ms/step - loss: 2.5825 - categorical_accuracy: 0.2103 - val_loss: 2.5658 - val_categorical_accuracy: 0.1971
Epoch 7/500
39

In [9]:
# take model predictions
res3 = model3.predict(x_test)
y_true = np.argmax(y_test, axis = 1).tolist()
y_pred3 = np.argmax(res3, axis = 1).tolist()
print(f'Accuracy: {round(accuracy_score(y_true, y_pred3)*100, 1)}%')

if accuracy_score(y_true, y_pred3) >= 0.70:
    model3.save('lstm_model3.h5', save_format = 'h5')
else:
    print('Model accuracy insufficient')

25/25 [==============================] - 2s 36ms/step
Accuracy: 66.4%
Model accuracy insufficient


### Variant 3

In [8]:
model4 = choose_lstm(6)
opt = keras.optimizers.Adam(learning_rate = 0.001)
model4.compile(optimizer = opt, loss = "categorical_crossentropy", metrics = ['categorical_accuracy'])
model4.summary()
model4.fit(x_train, y_train, epochs = 500, batch_size = 64, validation_split = 0.2, callbacks = [term, tb_callback, early])

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_2 (LSTM)               (None, 321, 128)          181248    
                                                                 
 dropout_2 (Dropout)         (None, 321, 128)          0         
                                                                 
 bidirectional_1 (Bidirecti  (None, 128)               98816     
 onal)                                                           
                                                                 
 dropout_3 (Dropout)         (None, 128)               0         
                                                                 
 dense_1 (Dense)             (None, 26)                3354      
                                                                 
Total params: 283418 (1.08 MB)
Trainable params: 283418 (1.08 MB)
Non-trainable params: 0 (0.00 Byte)
__________________

In [9]:
# take model predictions
res4 = model4.predict(x_test)
y_true = np.argmax(y_test, axis = 1).tolist()
y_pred4 = np.argmax(res4, axis = 1).tolist()
print(f'Accuracy: {round(accuracy_score(y_true, y_pred4)*100, 1)}%')

if accuracy_score(y_true, y_pred4) >= 0.70:
    model4.save('lstm_model4.h5', save_format = 'h5')
else:
    print('Model accuracy insufficient')

25/25 [==============================] - 2s 24ms/step
Accuracy: 73.2%


/home/zhwei/anaconda3/envs/slproj/lib/python3.9/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


# Apply whole data to model

In [3]:
x = np.concatenate((x_train, x_test), axis = 0)
y = np.concatenate((y_train, y_test), axis = 0)

In [7]:
early = EarlyStopping(monitor = 'val_loss', patience = 200)
checkpoint = ModelCheckpoint("lstm_model_best.h5", monitor = 'val_loss', verbose = 1, save_best_only = True, mode = 'min')

In [8]:
model = choose_lstm(4)
opt = keras.optimizers.Adam(learning_rate = 0.001)
model.compile(optimizer = opt, loss = "categorical_crossentropy", metrics = ['categorical_accuracy'])
model.summary()
model.fit(x, y, epochs = 500, batch_size = 64, validation_split = 0.2, callbacks = [term, tb_callback, early, checkpoint])

2024-03-29 21:57:37.951471: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-29 21:57:38.183929: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-29 21:57:38.183980: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-29 21:57:38.190799: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-29 21:57:38.190893: I external/local_xla/xla/stream_executor

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 321, 128)          181248    
                                                                 
 dropout (Dropout)           (None, 321, 128)          0         
                                                                 
 bidirectional (Bidirection  (None, 321, 128)          98816     
 al)                                                             
                                                                 
 dropout_1 (Dropout)         (None, 321, 128)          0         
                                                                 
 bidirectional_1 (Bidirecti  (None, 64)                41216     
 onal)                                                           
                                                                 
 dropout_2 (Dropout)         (None, 64)                0

2024-03-29 21:57:55.186685: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8904
2024-03-29 21:57:56.071277: I external/local_xla/xla/service/service.cc:168] XLA service 0x7f03a401d830 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-03-29 21:57:56.071317: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 3060 Laptop GPU, Compute Capability 8.6
2024-03-29 21:57:56.109996: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1711720676.252815  275309 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


49/49 [==============================] - ETA: 0s - loss: 3.1270 - categorical_accuracy: 0.0908
Epoch 1: val_loss improved from inf to 2.91544, saving model to lstm_model_best.h5
49/49 [==============================] - 15s 129ms/step - loss: 3.1270 - categorical_accuracy: 0.0908 - val_loss: 2.9154 - val_categorical_accuracy: 0.1220
Epoch 2/500


/home/zhwei/anaconda3/envs/slproj/lib/python3.9/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


49/49 [==============================] - ETA: 0s - loss: 2.8340 - categorical_accuracy: 0.1508
Epoch 2: val_loss improved from 2.91544 to 2.69917, saving model to lstm_model_best.h5
49/49 [==============================] - 5s 97ms/step - loss: 2.8340 - categorical_accuracy: 0.1508 - val_loss: 2.6992 - val_categorical_accuracy: 0.1746
Epoch 3/500
49/49 [==============================] - ETA: 0s - loss: 2.6636 - categorical_accuracy: 0.1849
Epoch 3: val_loss improved from 2.69917 to 2.58229, saving model to lstm_model_best.h5
49/49 [==============================] - 5s 94ms/step - loss: 2.6636 - categorical_accuracy: 0.1849 - val_loss: 2.5823 - val_categorical_accuracy: 0.2080
Epoch 4/500
49/49 [==============================] - ETA: 0s - loss: 2.5730 - categorical_accuracy: 0.1881
Epoch 4: val_loss improved from 2.58229 to 2.53845, saving model to lstm_model_best.h5
49/49 [==============================] - 5s 93ms/step - loss: 2.5730 - categorical_accuracy: 0.1881 - val_loss: 2.5384 - v

In [9]:
model.save('lstm_model_final.h5', save_format = 'h5')

In [4]:
model = keras.models.load_model('lstm_model_final.h5')
res = model.predict(x)
y_true = np.argmax(y, axis = 1).tolist()
y_pred = np.argmax(res, axis = 1).tolist()
print(f'Accuracy: {round(accuracy_score(y_true, y_pred)*100, 1)}%')

122/122 [==============================] - 14s 106ms/step
Accuracy: 94.1%


In [5]:
model_best = keras.models.load_model('lstm_model_best.h5')
res = model_best.predict(x)
y_true = np.argmax(y, axis = 1).tolist()
y_pred = np.argmax(res, axis = 1).tolist()
print(f'Accuracy: {round(accuracy_score(y_true, y_pred)*100, 1)}%')

122/122 [==============================] - 14s 104ms/step
Accuracy: 88.8%


# Hyperparameter Tuning (Cross-validation)

In [ ]:
# from scikeras.wrappers import KerasClassifier
# from sklearn.model_selection import GridSearchCV

In [ ]:
# def create_lstm_model(units = 64, dropout_rate = 0.5, l2_strength = 0.01, learning_rate=0.001):
#     model = Sequential()
#     model.add(LSTM(units, return_sequences = True, input_shape = input_shape))
#     model.add(Dropout(dropout_rate))
#     model.add(Bidirectional(LSTM(units, return_sequences = True)))
#     model.add(Dropout(dropout_rate))
#     model.add(Bidirectional(LSTM(units)))
#     model.add(Dropout(dropout_rate))
#     model.add(Dense(actions.shape[0], activation = "softmax"))
    
#     opt = keras.optimizers.Adam(learning_rate = learning_rate)
#     model.compile(optimizer = opt, loss = "categorical_crossentropy", metrics = ['categorical_accuracy'])
    
#     return model

# lstm_model = KerasClassifier(build_fn = create_lstm_model, epochs = 800, batch_size = 32, validation_split = 0.2, verbose = 1)

# # hyperparameters
# param_grid = {
#     'units': [64, 128],  
#     'dropout_rate': [0.2, 0.5],  
#     'l2_strength': [0.001, 0.01], 
#     'learning_rate': [0.001, 0.01]   
# }

# # grid search
# grid = GridSearchCV(estimator=lstm_model, param_grid=param_grid, cv=3, verbose=2)
# grid_result = grid.fit(x_train, y_train, callbacks = [term, tb_callback, early]) 

# # best parameters and best accuracy
# print("Best parameters found: ", grid_result.best_params_)
# print("Best accuracy found: ", grid_result.best_score_)